# Loading data

In [1]:
import pandas as pd

## Loading metrics

In [2]:
androidListSamples = pd.read_csv("..\\..\\1-GettingQuestions\\samplesWithQuestions\\androidSamples.csv")
awsListSamples = pd.read_csv("..\\..\\1-GettingQuestions\\samplesWithQuestions\\awsSamples.csv")
azureListSamples = pd.read_csv("..\\..\\1-GettingQuestions\\samplesWithQuestions\\azureSamples.csv")
springListSamples = pd.read_csv("..\\..\\1-GettingQuestions\\samplesWithQuestions\\springSamples.csv")

### 159 Code Samples

In [3]:
def getSampleDataframe(name):
    name = name.replace("/", "\\")
    sample = pd.read_csv("..\\..\\2-ExtractingMetrics\\metrics\\"+name+".csv", parse_dates=True)
    sampleAverage = sample[sample["type"] == "average"]
    sampleAverage["numberJavaFiles"] = sample[sample["type"] == "absolute"]["numberJavaFiles"].values
    sampleAverage["CountDeclClass"] = sample[sample["type"] == "absolute"]["CountDeclClass"].values
    sampleAverage["readability"] = (sampleAverage["readability"] * sampleAverage["numberJavaFiles"]) / sampleAverage["CountDeclClass"]
    sampleAverage['commitDate'] = pd.to_datetime(sampleAverage['commitDate'].astype(str).str[:-6])
    sampleAverage.set_index("commitDate", inplace=True)
    sampleAverage.dropna(axis=1, how="all", inplace=True)
    sampleAverage = sampleAverage.groupby([(sampleAverage.index.year), (sampleAverage.index.month)]).mean()
    generic = pd.DataFrame(index=pd.date_range(start="2013-04-15", end="2020-09-30", freq="M", name="commitDate"))
    generic = generic.groupby([(generic.index.year), (generic.index.month)]).sum()
    return pd.concat([generic, sampleAverage], axis=1).fillna(method='ffill').dropna(how="all")

In [4]:
androidSamples = pd.concat([getSampleDataframe(sample) for sample in androidListSamples["path"]], axis=0)
awsSamples = pd.concat([getSampleDataframe(sample) for sample in awsListSamples["path"]], axis=0)
azureSamples = pd.concat([getSampleDataframe(sample) for sample in azureListSamples["path"]], axis=0)
springSamples = pd.concat([getSampleDataframe(sample) for sample in springListSamples["path"]], axis=0)

<ipython-input-3-153a14f4a358>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampleAverage["numberJavaFiles"] = sample[sample["type"] == "absolute"]["numberJavaFiles"].values
<ipython-input-3-153a14f4a358>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampleAverage["CountDeclClass"] = sample[sample["type"] == "absolute"]["CountDeclClass"].values
<ipython-input-3-153a14f4a358>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

In [5]:
androidSamples = androidSamples.groupby(level=[0,1]).mean()
awsSamples = awsSamples.groupby(level=[0,1]).mean()
azureSamples = azureSamples.groupby(level=[0,1]).mean()
springSamples = springSamples.groupby(level=[0,1]).mean()

Join all framework in one DataFrame

In [6]:
allMetricsAverage = pd.concat([androidSamples, awsSamples, azureSamples, springSamples], axis=0)

In [7]:
allMetrics = allMetricsAverage

In [8]:
allMetrics = allMetrics.groupby(level=[0,1]).mean()

In [9]:
stats = pd.DataFrame(index = allMetrics.min().index)

### Min

In [10]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(allMetrics.min())
stats["min"] = allMetrics.min()

AvgCyclomatic                     1.189702
AvgCyclomaticModified             1.189702
AvgCyclomaticStrict               1.189702
AvgEssential                      0.951672
AvgLine                           9.531696
AvgLineBlank                      1.070659
AvgLineCode                       7.715891
AvgLineComment                    0.419500
CountClassBase                    1.000000
CountClassCoupled                 3.264992
CountClassCoupledModified         0.433466
CountClassDerived                 0.000000
CountDeclClass                    2.672043
CountDeclClassMethod              0.634077
CountDeclClassVariable            0.030187
CountDeclInstanceMethod           0.566468
CountDeclInstanceVariable         0.139172
CountDeclMethod                   1.306548
CountDeclMethodAll               13.306548
CountDeclMethodDefault            0.064095
CountDeclMethodPrivate            0.000000
CountDeclMethodProtected          0.009524
CountDeclMethodPublic             1.073129
CountLine  

### Max

In [11]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(allMetrics.max())
stats["max"] = allMetrics.max()

AvgCyclomatic                      1.927815
AvgCyclomaticModified              1.903504
AvgCyclomaticStrict                1.970422
AvgEssential                       1.085961
AvgLine                           24.621755
AvgLineBlank                       3.508371
AvgLineCode                       17.253979
AvgLineComment                     3.991445
CountClassBase                     1.342121
CountClassCoupled                  7.786287
CountClassCoupledModified          1.144744
CountClassDerived                  0.054412
CountDeclClass                    31.530925
CountDeclClassMethod               4.537838
CountDeclClassVariable             1.517573
CountDeclInstanceMethod            3.264868
CountDeclInstanceVariable          1.641126
CountDeclMethod                    7.633255
CountDeclMethodAll                21.802844
CountDeclMethodDefault             0.268231
CountDeclMethodPrivate             1.761447
CountDeclMethodProtected           0.230213
CountDeclMethodPublic           

### Average

In [12]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(allMetrics.mean())
stats["avg"] = allMetrics.mean()

AvgCyclomatic                      1.673810
AvgCyclomaticModified              1.651199
AvgCyclomaticStrict                1.723968
AvgEssential                       1.025549
AvgLine                           16.147384
AvgLineBlank                       1.987497
AvgLineCode                       11.422824
AvgLineComment                     2.458739
CountClassBase                     1.186117
CountClassCoupled                  6.727829
CountClassCoupledModified          0.887791
CountClassDerived                  0.020353
CountDeclClass                    18.477604
CountDeclClassMethod               2.876729
CountDeclClassVariable             1.222358
CountDeclInstanceMethod            2.822038
CountDeclInstanceVariable          1.318095
CountDeclMethod                    5.698767
CountDeclMethodAll                18.574352
CountDeclMethodDefault             0.165823
CountDeclMethodPrivate             1.227385
CountDeclMethodProtected           0.173100
CountDeclMethodPublic           

### Median

In [13]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(allMetrics.median())
stats["med"] = allMetrics.median()

AvgCyclomatic                      1.665803
AvgCyclomaticModified              1.640891
AvgCyclomaticStrict                1.724813
AvgEssential                       1.015502
AvgLine                           16.964489
AvgLineBlank                       1.941244
AvgLineCode                       12.167654
AvgLineComment                     2.668622
CountClassBase                     1.152102
CountClassCoupled                  6.914571
CountClassCoupledModified          0.976668
CountClassDerived                  0.009973
CountDeclClass                    20.393294
CountDeclClassMethod               2.531457
CountDeclClassVariable             1.276721
CountDeclInstanceMethod            3.062066
CountDeclInstanceVariable          1.462045
CountDeclMethod                    5.641019
CountDeclMethodAll                18.159749
CountDeclMethodDefault             0.155084
CountDeclMethodPrivate             1.100915
CountDeclMethodProtected           0.208416
CountDeclMethodPublic           

### Standard Deviation

In [14]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(allMetrics.std())
stats["std"] = allMetrics.std()

AvgCyclomatic                     0.107471
AvgCyclomaticModified             0.112249
AvgCyclomaticStrict               0.104545
AvgEssential                      0.031401
AvgLine                           3.351460
AvgLineBlank                      0.486537
AvgLineCode                       2.344248
AvgLineComment                    0.708100
CountClassBase                    0.073399
CountClassCoupled                 0.688970
CountClassCoupledModified         0.214021
CountClassDerived                 0.017911
CountDeclClass                    9.149877
CountDeclClassMethod              1.041942
CountDeclClassVariable            0.201805
CountDeclInstanceMethod           0.533292
CountDeclInstanceVariable         0.371440
CountDeclMethod                   1.386701
CountDeclMethodAll                2.125264
CountDeclMethodDefault            0.065671
CountDeclMethodPrivate            0.346874
CountDeclMethodProtected          0.067003
CountDeclMethodPublic             1.449999
CountLine  

In [15]:
stats.to_csv("stats.csv")

Questions

## Loading SO questions

In [16]:
androidQuestions = pd.read_csv("..\\..\\1-GettingQuestions\\questions\\androidQuestions.csv")
awsQuestions = pd.read_csv("..\\..\\1-GettingQuestions\\questions\\awsQuestions.csv")
azureQuestions = pd.read_csv("..\\..\\1-GettingQuestions\\questions\\azureQuestions.csv")
springQuestions = pd.read_csv("..\\..\\1-GettingQuestions\\questions\\springQuestions.csv")

In [17]:
allQuestions = pd.concat([androidQuestions, awsQuestions, azureQuestions, springQuestions], axis=0, ignore_index=True)

In [18]:
allQuestions['creationDate'] = pd.to_datetime(allQuestions['creationDate'])
allQuestions.sort_values(by="creationDate", inplace=True)
allQuestions.set_index(allQuestions['creationDate'], inplace=True)
allQuestions["questions"] = 1
del allQuestions["ownerUserId"]
del allQuestions["postTypeId"]
del allQuestions["acceptedAnswerId"]
del allQuestions["id"]
del allQuestions['creationDate']
allQuestions=allQuestions.groupby([(allQuestions.index.year), (allQuestions.index.month)]).sum()
allQuestions

questions
creationDate creationDate           
2013         11                    2
2014         1                     1
             3                     1
             4                     4
             5                     1
...                              ...
2020         4                    10
             5                     8
             6                     4
             7                     8
             8                     8

[80 rows x 1 columns]

In [19]:
allQuestions["questions"]=allQuestions.cumsum()

In [20]:
allQuestions

questions
creationDate creationDate           
2013         11                    2
2014         1                     3
             3                     4
             4                     8
             5                     9
...                              ...
2020         4                  1269
             5                  1277
             6                  1281
             7                  1289
             8                  1297

[80 rows x 1 columns]

## Merging the data

In [21]:
metricsAndQuestions = pd.concat([allMetrics, allQuestions], axis=1)
metricsAndQuestions["questions"].fillna(0, inplace=True)
metricsAndQuestions.fillna(method="ffill", inplace=True)
metricsAndQuestions.dropna(inplace=True)

In [22]:
metricsAndQuestions = metricsAndQuestions.drop((2020, 9))

## Saving

In [23]:
metricsAndQuestions.to_csv("metricsAndQuestions.csv")

In [24]:
pd.read_csv("metricsAndQuestions.csv", index_col=[0,1])

AvgCyclomatic  AvgCyclomaticModified  AvgCyclomaticStrict  \
2013 4       1.618339               1.618339             1.618339   
     5       1.189702               1.189702             1.189702   
     6       1.832684               1.832684             1.832684   
     7       1.796627               1.796627             1.812500   
     8       1.798722               1.798722             1.813873   
...               ...                    ...                  ...   
2020 4       1.611153               1.577891             1.698980   
     5       1.621970               1.588709             1.709739   
     6       1.620074               1.587153             1.707627   
     7       1.619861               1.587026             1.707405   
     8       1.653771               1.620935             1.743896   

        AvgEssential    AvgLine  AvgLineBlank  AvgLineCode  AvgLineComment  \
2013 4      0.951672  15.659580      2.371032    12.145692        1.214286   
     5      0.953730   9.531696      1.231337     7.932814        0.419500   
     6      0.996131  12.513934      2.150244     8.768101        1.360958   
     7      1.000000  11.959005      2.002518     8.363965        1.327381   
     8      1.000000  11.931189      1.997585     8.346462        1.323864   
...              ...        ...           ...          ...             ...   
2020 4      1.076477  16.807571      1.815383    12.057656        2.632252   
     5      1.076442  16.887538      1.818227    12.130030        2.637039   
     6      1.075422  16.931315      1.814126    12.169351        2.652578   
     7      1.075380  16.925086      1.813402    12.165956        2.652274   
     8      1.075380  17.151695      1.817679    12.382208        2.656773   

        CountClassBase  CountClassCoupled  ...  PercentLackOfCohesion  \
2013 4        1.000000           4.066893  ...               0.996173   
     5        1.035606           3.264992  ...               5.499265   
     6        1.261905           7.130366  ...              24.050000   
     7        1.265873           7.291667  ...              26.712302   
     8        1.264205           7.252841  ...              26.613636   
...                ...                ...  ...                    ...   
2020 4        1.143503           6.998108  ...              29.009687   
     5        1.143474           7.036406  ...              28.943180   
     6        1.139911           7.040763  ...              29.268200   
     7        1.139860           7.040809  ...              29.246863   
     8        1.139562           7.146740  ...              29.082851   

        PercentLackOfCohesionModified  RatioCommentToCode  SumCyclomatic  \
2013 4                       0.996173            0.052780       2.163690   
     5                       5.499265            0.056063       2.176760   
     6                      24.050000            0.193954       8.440864   
     7                      26.712302            0.191987       8.594582   
     8                      26.575758            0.192975       8.596922   
...                               ...                 ...            ...   
2020 4                      25.457074            0.307560      17.559135   
     5                      25.389795            0.308499      17.597126   
     6                      25.681186            0.308096      17.640748   
     7                      25.663714            0.307928      17.639400   
     8                      25.499501            0.309246      17.704841   

        SumCyclomaticModified  SumCyclomaticStrict  SumEssential  \
2013 4               2.163690             2.163690      1.354167   
     5               2.176760             2.176760      1.738345   
     6               8.440864             8.440864      4.756575   
     7               8.594582             8.618391      4.876786   
     8               8.596922             8.619650      4.887642   
...                       ...               